In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

## 데이터 둘러보기
data_path = '/content/drive/MyDrive/'

train = pd.read_csv(data_path + 'train.csv')
test = pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sampleSubmission.csv')

### 데이터 합치기
all_data_temp = pd.concat([train, test], ignore_index=True)
print(all_data_temp)

In [ ]:
### 필요 없는 피처 제거
### casual + registered = count
### count : target
drop_features = ['casual','registered', 'count']

all_data = all_data_temp.drop(drop_features, axis=1)

all_data

In [ ]:
all_data.info()

In [ ]:
# 문자열의 배열을 판다스 Timestamp로 변환
dates = all_data['datetime']
print(dates)
print('\n')

ts_dates = pd.to_datetime(dates)
print(ts_dates)


In [ ]:
all_data['new_Date'] = pd.to_datetime(all_data['datetime'])
all_data['new_Year'] = all_data['new_Date'].dt.year
all_data['new_Month'] = all_data['new_Date'].dt.month
all_data['new_Day'] = all_data['new_Date'].dt.day

all_data['new_Hour'] = all_data['new_Date'].dt.hour
all_data['new_Minute'] = all_data['new_Date'].dt.minute
all_data['new_Second'] = all_data['new_Date'].dt.second


print(all_data.head())
print('\n')

print(all_data.info())
print('\n')


In [ ]:
## 데이터 시각화
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
#추가하면 matplotlib이 그린 그래프를 주피터 노트북에서 바로 출력

In [ ]:
### 분포도
# Step 1
### Target 값인 count 칼럼 정균 분포 이루는지 확인
### 왜곡 존재, 로그를 적용해서 변환 필요
mpl.rc('font', size=15)

sns.displot(train['count']);


In [ ]:
### 왜곡된 값을 정규 분포 형태로 바꿈
sns.displot(np.log(train['count']))

In [ ]:
### 막대 그래프
# Step 1
mpl.rc('font', size=10)
mpl.rc('axes', titlesize=15)
figure, axes = plt.subplots(nrows=3, ncols=2)
plt.tight_layout()
figure.set_size_inches(12,10)

#axes

# Step 2
sns.barplot(x='new_Year', y=train['count'], data=all_data, ax=axes[0,0])
sns.barplot(x='new_Month', y=train['count'], data=all_data, ax=axes[0,1])
sns.barplot(x='new_Day', y=train['count'], data=all_data, ax=axes[1,0])
sns.barplot(x='new_Hour', y=train['count'], data=all_data, ax=axes[1,1])
sns.barplot(x='new_Minute', y=train['count'], data=all_data, ax=axes[2,0])
sns.barplot(x='new_Second', y=train['count'], data=all_data, ax=axes[2,1])

# Step 3
axes[0,0].set(title='Rental amounts by year')
axes[0,1].set(title='Rental amounts by month')
axes[1,0].set(title='Rental amounts by day')
axes[1,1].set(title='Rental amounts by hour')
axes[2,0].set(title='Rental amounts by minute')
axes[2,1].set(title='Rental amounts by second')

axes[1,0].tick_params(axis='x', labelrotation=0)
axes[1,1].tick_params(axis='x', labelrotation=0)

In [ ]:
all_data['season'] = all_data['season'].map({1: 'Spring',
                                      2: 'Summer',
                                      3: 'Fall',
                                      4: 'Winter'})

all_data['weather'] = all_data['weather'].map({1: 'Clear',
                                      2: 'Mist, Few clouds',
                                      3: 'Light Snow, Rain',
                                      4: 'Heavy Rain, Thunderstorm, Snow, Fog'})

print(all_data.head())
print('\n')

In [ ]:
### 포인트 플롯
# Step 1
mpl.rc('font', size=11)
figure, axes = plt.subplots(nrows=4)
figure.set_size_inches(12,18)

# Step 2
### 평일, 주말, 계절, 날씨에 따른 target값 표현
sns.pointplot(x='new_Hour', y=train['count'], data=all_data, hue='workingday', ax=axes[0])
sns.pointplot(x='new_Hour', y=train['count'], data=all_data, hue='holiday', ax=axes[1])
#sns.pointplot(x='new_Hour', y='count', data=train, hue='weekday', ax=axes[2])
sns.pointplot(x='new_Hour', y=train['count'], data=all_data, hue='season', ax=axes[2])
sns.pointplot(x='new_Hour', y=train['count'], data=all_data, hue='weather', ax=axes[3])

In [ ]:
train[['temp', 'atemp', 'humidity', 'windspeed', 'count']].corr()


#corrMat = all_data[['temp', 'atemp', 'humidity', 'windspeed', train['count']].corr()

#fig, ax = plt.subplots()
#fig.set_size_inches(10, 10)

#sns.heatmap(corrMat, annot=True)

#ax.set(title='Heatmap of Numerical Data');

In [ ]:
### 피처 엔지니어링
# 이상치 제거
all_data = all_data[all_data['weather'] != 4]

#
print(train.shape, test.shape)
print('\n')

### 데이터 합치기
#all_data_temp = pd.concat([train, test], ignore_index=True)
print(all_data)

#
all_data['season'] = all_data['season'].map({'Spring': 1,
                                      'Summer': 2,
                                      'Fall': 3,
                                      'Winter': 4})

all_data['weather'] = all_data['weather'].map({'Clear': 1,
                                      'Mist, Few clouds': 2,
                                      'Light Snow, Rain': 3,
                                      'Heavy Rain, Thunderstorm, Snow, Fog': 4})

In [ ]:
### 필요 없는 피처 제거
drop_features = ['datetime','new_Date','windspeed','new_Day','new_Minute','new_Second']

all_data = all_data.drop(drop_features, axis=1)

all_data

In [ ]:
### 인코딩
from sklearn.preprocessing import OneHotEncoder

onehot_encoder = OneHotEncoder()

onehot_features = ['season', 'weather', 'new_Year', 'new_Month', 'new_Hour']

encoded_features_matrix = onehot_encoder.fit_transform(all_data[onehot_features])

encoded_features_matrix


In [ ]:
### 정규화
from sklearn.preprocessing import MinMaxScaler

normalized_features = ['temp', 'atemp', 'humidity']

minmax_matrix = MinMaxScaler().fit_transform(all_data[normalized_features])

minmax_matrix

In [ ]:
from scipy import sparse

#all_data_sprs = sparse.hstack([sparse.csr_matrix(all_data), encoded_features_matrix], format='csr')

all_data_sprs = sparse.hstack([encoded_features_matrix, minmax_matrix], format='csr')


all_data_sprs


In [ ]:
#　데이터 나누기
num_train = len(train)

X_train = all_data_sprs[:num_train]
X_test = all_data_sprs[num_train:]

# 타깃값 count 제거
#X_train = X_train.drop(['count'], axis=1)
#X_test = X_test.drop(['count'], axis=1)

y = train['count']

print(X_train.shape, y.shape)


In [ ]:
## 평가지표 계산 함수
## 값이 작을수록 회귀 성능 Good
import numpy as np

def rmsle(y_true, y_pred, convertExp=True):
    if convertExp:
        y_true = np.exp(y_true)
        y_pred = np.exp(y_pred)

    log_true = np.nan_to_num(np.log(y_true+1))
    log_pred = np.nan_to_num(np.log(y_pred+1))

    output = np.sqrt(np.mean((log_true - log_pred)**2))

    return output

In [ ]:
# ## 베이스라인 모델 훈련
# from sklearn.linear_model import LinearRegression

# linear_reg_model = LinearRegression()

# log_y = np.log(y)

# linear_reg_model.fit(X_train, log_y)

# ## 모델 성능 검증
# preds = linear_reg_model.predict(X_train)

# print(f'선형 회귀의 RMSLE 값 : {rmsle(log_y, preds, True): .4f}')


In [ ]:
# ## 성능 개선 1 : 릿지 회귀 모델
# ### 모델 생성
# from sklearn.linear_model import Ridge
# from sklearn.model_selection import GridSearchCV
# from sklearn import metrics

# ridge_model = Ridge()

# ### 그리드서치 객체 생성
# # 하이퍼파라미터 값 목록
# ridge_params ={'max_iter':[3000], 'alpha':[0.01, 0.1,1,2,3,4,10,30,100,200,300,400,800,900,1000]}

# # 교차 검증용 평가 함수 (RMSLE 점수 계산)
# rmsle_scorer = metrics.make_scorer(rmsle, greater_is_better=False)

# # 그리드서치 객체 생성
# gridsearch_ridge_model = GridSearchCV(estimator=ridge_model,
#                                      param_grid=ridge_params,
#                                      scoring=rmsle_scorer,
#                                      cv=5)

# ### 그리드서치 수행
# log_y = np.log(y)
# gridsearch_ridge_model.fit(X_train, log_y)

# print('최적 하이퍼파라미터 :', gridsearch_ridge_model.best_params_)

In [ ]:
# ## 성능 검증
# preds = gridsearch_ridge_model.best_estimator_.predict(X_train)

# print(f'릿지 회귀의 RMSLE 값: {rmsle(log_y, preds, True): .4f}')

In [ ]:
# ## 성능 개선 2 : 라쏘 회귀 모델
# ### 모델 생성
# from sklearn.linear_model import Lasso
# from sklearn.model_selection import GridSearchCV
# from sklearn import metrics

# lasso_model = Lasso()

# ### 그리드서치 객체 생성
# # 하이퍼파라미터 값 목록
# lasso_alpha = 1/np.array([0.1,1,2,3,4,10,30,100,200,300,400,800,900,1000])
# lasso_params = {'max_iter':[3000], 'alpha': lasso_alpha}

# # 교차 검증용 평가 함수 (RMSLE 점수 계산)
# rmsle_scorer = metrics.make_scorer(rmsle, greater_is_better=False)

# # 그리드서치 객체 생성
# gridsearch_lasso_model = GridSearchCV(estimator=lasso_model,
#                                      param_grid=lasso_params,
#                                      scoring=rmsle_scorer,
#                                      cv=5)

# ### 그리드서치 수행
# log_y = np.log(y)
# gridsearch_lasso_model.fit(X_train, log_y)

# print('최적 하이퍼파라미터 :', gridsearch_lasso_model.best_params_)

In [ ]:
# ## 성능 검증
# preds = gridsearch_lasso_model.best_estimator_.predict(X_train)

# print(f'라쏘 회귀의 RMSLE 값: {rmsle(log_y, preds, True): .4f}')

## Start

In [ ]:
## 피처 엔지니어링
print(train['datetime'])
print('\n')
print(train['datetime'][10881])
print('\n')
print(train['datetime'][10881].split())
print('\n')

print(train['datetime'][10881].split()[0])
print(train['datetime'][10881].split()[1])

In [ ]:
print(train['datetime'][10881].split()[0])

print(train['datetime'][10881].split()[0].split("-"))

print(train['datetime'][10881].split()[0].split("-")[0])
print(train['datetime'][10881].split()[0].split("-")[1])
print(train['datetime'][10881].split()[0].split("-")[2])

In [ ]:
print(train['datetime'][10881].split()[1])

print(train['datetime'][10881].split()[1].split(":"))

print(train['datetime'][10881].split()[1].split(":")[0])
print(train['datetime'][10881].split()[1].split(":")[1])
print(train['datetime'][10881].split()[1].split(":")[2])

In [ ]:
train['date'] = train['datetime'].apply(lambda x: x.split()[0])

train['year'] = train['datetime'].apply(lambda x: x.split()[0].split('-')[0])
train['month'] = train['datetime'].apply(lambda x: x.split()[0].split('-')[1])
train['day'] = train['datetime'].apply(lambda x: x.split()[0].split('-')[2])

train['hour'] = train['datetime'].apply(lambda x: x.split()[1].split(':')[0])
train['minute'] = train['datetime'].apply(lambda x: x.split()[1].split(':')[1])
train['second'] = train['datetime'].apply(lambda x: x.split()[1].split(':')[2])

In [ ]:
from datetime import datetime
import calendar

print(train['date'][10881])
print(datetime.strptime(train['date'][10881],'%Y-%m-%d'))
print(datetime.strptime(train['date'][10881],'%Y-%m-%d').weekday())

print(calendar.day_name[datetime.strptime(train['date'][10881],'%Y-%m-%d').weekday()])

In [ ]:
train['weekday'] = train['date'].apply(lambda dateString: calendar.day_name[datetime.strptime(dateString, "%Y-%m-%d").weekday()])

In [ ]:
train['season'] = train['season'].map({1: 'Spring',
                                      2: 'Summer',
                                      3: 'Fall',
                                      4: 'Winter'})

train['weather'] = train['weather'].map({1: 'Clear',
                                      2: 'Mist, Few clouds',
                                      3: 'Light Snow, Rain',
                                      4: 'Heavy Rain, Thunderstorm, Snow, Fog'})

In [ ]:
train.info()

In [ ]:
## 데이터 시각화
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
#추가하면 matplotlib이 그린 그래프를 주피터 노트북에서 바로 출력

In [ ]:
### 분포도
mpl.rc('font', size=15)

sns.displot(train['count']);

In [ ]:
sns.displot(np.log(train['count']));

In [ ]:
### 막대 그래프
# Step 1
mpl.rc('font', size=10)
mpl.rc('axes', titlesize=15)
figure, axes = plt.subplots(nrows=3, ncols=2)
plt.tight_layout()
figure.set_size_inches(12,10)

#axes

# Step 2
sns.barplot(x='year', y='count', data=train, ax=axes[0,0])
sns.barplot(x='month', y='count', data=train, ax=axes[0,1])
sns.barplot(x='day', y='count', data=train, ax=axes[1,0])
sns.barplot(x='hour', y='count', data=train, ax=axes[1,1])
sns.barplot(x='minute', y='count', data=train, ax=axes[2,0])
sns.barplot(x='second', y='count', data=train, ax=axes[2,1])

# Step 3
axes[0,0].set(title='Rental amounts by year')
axes[0,1].set(title='Rental amounts by month')
axes[1,0].set(title='Rental amounts by day')
axes[1,1].set(title='Rental amounts by hour')
axes[2,0].set(title='Rental amounts by minute')
axes[2,1].set(title='Rental amounts by second')

axes[1,0].tick_params(axis='x', labelrotation=0)
axes[1,1].tick_params(axis='x', labelrotation=0)

In [ ]:
### 박스 플롯
# Step 1
mpl.rc('font', size=12)
figure, axes = plt.subplots(nrows=2, ncols=2)
plt.tight_layout()
figure.set_size_inches(15,10)

# Step 2
sns.boxplot(x='season', y='count', data=train, ax=axes[0,0])
sns.boxplot(x='weather', y='count', data=train, ax=axes[0,1])
sns.boxplot(x='holiday', y='count', data=train, ax=axes[1,0])
sns.boxplot(x='workingday', y='count', data=train, ax=axes[1,1])

# Step 3
axes[0,0].set(title='Box Plot On Count Across Season')
axes[0,1].set(title='Box Plot On Count Across Weather')
axes[1,0].set(title='Box Plot On Count Across Holiday')
axes[1,1].set(title='Box Plot On Count Across Working Day')

axes[0,1].tick_params(axis='x', labelrotation=10)

In [ ]:
### 포인트 플롯
# Step 1
mpl.rc('font', size=11)
figure, axes = plt.subplots(nrows=5)
figure.set_size_inches(12,18)

# Step 2
sns.pointplot(x='hour', y='count', data=train, hue='workingday', ax=axes[0])
sns.pointplot(x='hour', y='count', data=train, hue='holiday', ax=axes[1])
sns.pointplot(x='hour', y='count', data=train, hue='weekday', ax=axes[2])
sns.pointplot(x='hour', y='count', data=train, hue='season', ax=axes[3])
sns.pointplot(x='hour', y='count', data=train, hue='weather', ax=axes[4])

In [ ]:
### 회귀선을 포함한 산점도 그래프
# Step 1
mpl.rc('font', size=15)
figure, axes = plt.subplots(nrows=2, ncols=2)
plt.tight_layout()
figure.set_size_inches(7,6)

# Step 2
sns.regplot(x='temp', y='count', data=train, ax=axes[0,0], scatter_kws={'alpha':0.2}, line_kws={'color': 'blue'})
sns.regplot(x='atemp', y='count', data=train, ax=axes[0,1], scatter_kws={'alpha':0.2}, line_kws={'color': 'blue'})
sns.regplot(x='windspeed', y='count', data=train, ax=axes[1,0], scatter_kws={'alpha':0.2}, line_kws={'color': 'blue'})
sns.regplot(x='humidity', y='count', data=train, ax=axes[1,1], scatter_kws={'alpha':0.2}, line_kws={'color': 'blue'})

In [ ]:
### 히트맵
train[['temp','atemp','humidity','windspeed','count']].corr()

In [ ]:
corrMat = train[['temp','atemp','humidity','windspeed','count']].corr()

fig, ax = plt.subplots()
fig.set_size_inches(10, 10)

sns.heatmap(corrMat, annot=True)

ax.set(title='Heatmap of Numerical Data');

## Start

In [ ]:
## 베이스라인 모델
import pandas as pd
import numpy as np
from tqdm import tqdm

### 데이터 불러오기
data_path = '/content/drive/MyDrive/'

train = pd.read_csv(data_path + 'train.csv')
test = pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sampleSubmission.csv')

### 피처 엔지니어링
# 이상치 제거
train = train[train['weather'] != 4]

In [ ]:
### 데이터 합치기
all_data_temp = pd.concat([train, test])
all_data_temp

In [ ]:
all_data = pd.concat([train, test], ignore_index=True)
all_data

In [ ]:
### 파생 피처 추가
from datetime import datetime

all_data['date'] = all_data['datetime'].apply(lambda x: x.split()[0])

all_data['year'] = all_data['datetime'].apply(lambda x: x.split()[0].split('-')[0])

all_data['month'] = all_data['datetime'].apply(lambda x: x.split()[0].split('-')[1])

all_data['day'] = all_data['datetime'].apply(lambda x: x.split()[0].split('-')[2])

all_data['weekday'] = all_data['date'].apply(lambda dateString : datetime.strptime(dateString, "%Y-%m-%d").weekday())

all_data['hour'] = all_data['datetime'].apply(lambda x: x.split()[1].split(':')[0])


all_data

In [ ]:
### 필요 없는 피처 제거
drop_features = ['casual','registered','datetime','date','windspeed','month']

all_data = all_data.drop(drop_features, axis=1)

all_data

In [ ]:
## 데이터 나누기
X_train = all_data[~pd.isnull(all_data['count'])]
X_test = all_data[pd.isnull(all_data['count'])]

X_train

In [ ]:
# 타깃값 count 제거
X_train = X_train.drop(['count'], axis=1)
X_test = X_test.drop(['count'], axis=1)

y = train['count']

In [ ]:
X_train.head()

In [ ]:
X_train.info()

In [ ]:
## 평가지표 계산 함수
import numpy as np

def rmsle(y_true, y_pred, convertExp=True):
    if convertExp:
        y_true = np.exp(y_true)
        y_pred = np.exp(y_pred)

    log_true = np.nan_to_num(np.log(y_true+1))
    log_pred = np.nan_to_num(np.log(y_pred+1))

    output = np.sqrt(np.mean((log_true - log_pred)**2))

    return output

In [ ]:
## 모델 훈련
from sklearn.linear_model import LinearRegression

linear_reg_model = LinearRegression()

log_y = np.log(y)

linear_reg_model.fit(X_train, log_y)

In [ ]:
## 모델 성능 검증
preds = linear_reg_model.predict(X_train)

In [ ]:
print(f'선형 회귀의 RMSLE 값: {rmsle(log_y, preds, True): .4f}')

In [ ]:
## 예측 및 결과제출
linearreg_preds = linear_reg_model.predict(X_test)

#submission['count'] = np.exp(linearreg_preds)

#submission.to_csv('submission.csv', index=False)

linearreg_preds

In [ ]:
## 성능 개선 1 : 릿지 회귀 모델
### 모델 생성
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

ridge_model = Ridge()

### 그리드서치 객체 생성
# 하이퍼파라미터 값 목록
ridge_params ={'max_iter':[5000], 'alpha':[0.001,0.01,0.1,1,2,3,4,10,30,100,200,300,400,800,900,1000]}

# 교차 검증용 평가 함수 (RMSLE 점수 계산)
rmsle_scorer = metrics.make_scorer(rmsle, greater_is_better=False)

# 그리드서치 객체 생성
gridsearch_ridge_model = GridSearchCV(estimator=ridge_model,
                                     param_grid=ridge_params,
                                     scoring=rmsle_scorer,
                                     cv=5)

In [ ]:
### 그리드서치 수행
log_y = np.log(y)
gridsearch_ridge_model.fit(X_train, log_y)

print('최적 하이퍼파라미터 :', gridsearch_ridge_model.best_params_)

In [ ]:
## 성능 검증
preds = gridsearch_ridge_model.best_estimator_.predict(X_train)

print(f'릿지 회귀의 RMSLE 값: {rmsle(log_y, preds, True): .4f}')

In [ ]:
## 성능 개선 2 : 라쏘 회귀 모델
### 모델 생성
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

lasso_model = Lasso()

### 그리드서치 객체 생성
# 하이퍼파라미터 값 목록
lasso_alpha = 1/np.array([0.001,0.01,0.1,1,2,3,4,10,30,100,200,300,400,800,900,1000])
lasso_params = {'max_iter':[3000], 'alpha': lasso_alpha}

# 교차 검증용 평가 함수 (RMSLE 점수 계산)
rmsle_scorer = metrics.make_scorer(rmsle, greater_is_better=False)

# 그리드서치 객체 생성
gridsearch_lasso_model = GridSearchCV(estimator=lasso_model,
                                     param_grid=lasso_params,
                                     scoring=rmsle_scorer,
                                     cv=5)

### 그리드서치 수행
log_y = np.log(y)
gridsearch_lasso_model.fit(X_train, log_y)

print('최적 하이퍼파라미터 :', gridsearch_lasso_model.best_params_)

In [ ]:
## 성능 검증
preds = gridsearch_lasso_model.best_estimator_.predict(X_train)

print(f'라쏘 회귀의 RMSLE 값: {rmsle(log_y, preds, True): .4f}')

In [ ]:
## 성능 개선 3 : 랜덤 포레스트 회귀 모델

### 모델 생성
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

randomforest_model = RandomForestRegressor()

### 그리드서치 객체 생성
# 하이퍼파라미터 값 목록
rf_params = {'random_state':[15, 42],
             'n_estimators': [100,140,150,200,300,400]}
# 교차 검증용 평가 함수 (RMSLE 점수 계산)
rmsle_scorer = metrics.make_scorer(rmsle, greater_is_better=False)

# 그리드서치 객체 생성
gridsearch_random_forest_model = GridSearchCV(estimator=randomforest_model,
                                     param_grid=rf_params,
                                     scoring=rmsle_scorer,
                                     cv=5)

### 그리드서치 수행
log_y = np.log(y)
gridsearch_random_forest_model.fit(X_train, log_y)

print('최적 하이퍼파라미터 :', gridsearch_random_forest_model.best_params_)

In [ ]:
## 성능 검증
preds = gridsearch_random_forest_model.best_estimator_.predict(X_train)

print(f'랜덤 포레스트 회귀의 RMSLE 값: {rmsle(log_y, preds, True): .4f}')

In [ ]:
X_train['year'] = X_train['year'].astype(int)
X_train['day'] = X_train['day'].astype(int)
X_train['hour'] = X_train['hour'].astype(int)

In [ ]:
## 성능 개선 4 : XGBoost

### 모델 생성
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

xgb_model = XGBRegressor()

### 그리드서치 객체 생성
# 하이퍼파라미터 값 목록
xgb_params = {'random_state':[42],
              'subsample':[x/10 for x in range(6,10)],
              'colsample_bytree':[x/10.0 for x in range(6,10)]
            #  'n_estimators': [60,80,100,120,140,160],
              }
# 교차 검증용 평가 함수 (RMSLE 점수 계산)
rmsle_scorer = metrics.make_scorer(rmsle, greater_is_better=False)

# 그리드서치 객체 생성
gridsearch_xgb_model = GridSearchCV(estimator=xgb_model,
                                     param_grid=xgb_params,
                                     scoring=rmsle_scorer,
                                     cv=5)

### 그리드서치 수행
log_y = np.log(y)
gridsearch_xgb_model.fit(X_train, log_y)

print('최적 하이퍼파라미터 :', gridsearch_xgb_model.best_params_)

In [ ]:
## 성능 검증
preds = gridsearch_xgb_model.best_estimator_.predict(X_train)

print(f'XGBoost 회귀의 RMSLE 값: {rmsle(log_y, preds, True): .4f}')

In [ ]:
## 성능 개선 5 : LightGBM

### 모델 생성
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

lgbm_model = LGBMRegressor()

### 그리드서치 객체 생성
# 하이퍼파라미터 값 목록
lgbm_params = {'random_state':[42],
               'num_leaves':[32, 64],
               'n_estimators': [60,80,100,120,140,160]}

# 교차 검증용 평가 함수 (RMSLE 점수 계산)
rmsle_scorer = metrics.make_scorer(rmsle, greater_is_better=False)

# 그리드서치 객체 생성
gridsearch_lgbm_model = GridSearchCV(estimator=lgbm_model,
                                     param_grid=lgbm_params,
                                     scoring=rmsle_scorer,
                                     cv=5)

### 그리드서치 수행
log_y = np.log(y)
gridsearch_lgbm_model.fit(X_train, log_y)

print('최적 하이퍼파라미터 :', gridsearch_lgbm_model.best_params_)

In [ ]:
## 성능 검증
preds = gridsearch_lgbm_model.best_estimator_.predict(X_train)

print(f'LightGBM 회귀의 RMSLE 값: {rmsle(log_y, preds, True): .4f}')

In [ ]:
## 예측 및 결과 제출

import seaborn as sns
import matplotlib.pyplot as plt

randomforest_preds = gridsearch_random_forest_model.best_estimator_.predict(X_test)

figure, axes = plt.subplots(ncols=2)
figure.set_size_inches(10,4)

sns.histplot(y, bins=50, ax=axes[0])
sns.histplot(np.exp(randomforest_preds), bins=50, ax=axes[1])

axes[0].set_title('Train Data Distribution')
axes[1].set_title('Predicted Test Data Distribution')

In [ ]:
X_test['year'] = X_test['year'].astype(int)
X_test['day'] = X_test['day'].astype(int)
X_test['hour'] = X_test['hour'].astype(int)

In [ ]:
X_test.info()

In [ ]:
## 예측 및 결과 제출

import seaborn as sns
import matplotlib.pyplot as plt

xgb_preds = gridsearch_xgb_model.best_estimator_.predict(X_test)

figure, axes = plt.subplots(ncols=2)
figure.set_size_inches(10,4)

sns.histplot(y, bins=50, ax=axes[0])
sns.histplot(np.exp(xgb_preds), bins=50, ax=axes[1])

axes[0].set_title('Train Data Distribution')
axes[1].set_title('Predicted Test Data Distribution')

In [ ]:
## 예측 및 결과 제출

import seaborn as sns
import matplotlib.pyplot as plt

lgbm_preds = gridsearch_lgbm_model.best_estimator_.predict(X_test)

figure, axes = plt.subplots(ncols=2)
figure.set_size_inches(10,4)

sns.histplot(y, bins=50, ax=axes[0])
sns.histplot(np.exp(lgbm_preds), bins=50, ax=axes[1])

axes[0].set_title('Train Data Distribution')
axes[1].set_title('Predicted Test Data Distribution')

In [ ]:
## 결과 제출
# submission['count'] = np.exp(randomforest_preds)

# submission.to_csv('submission.csv', index=False)